In [1]:
import pandas as pd
import numpy as np
import datetime 
import re

In [2]:
# fr.head(20)

In [3]:
res=[[]]
gauge_list = []
nm_list = []
dicts = {}

fr = pd.read_excel('Унжа_данные.xlsx')
ind=fr.loc[fr.iloc[:,0] == "Код поста"].index
fy=int(fr.iloc[ind[0]+1,1])#первый год
for j in ind:#итерация по отдельным таблицам
    nm=[fr.iloc[j,1]]#код поста
    nm_list.append(nm)
    y=int(fr.iloc[j+1,1])#год
    gauge = [fr.iloc[j+2,1]]
    gauge_list.append(gauge)
    
    fd=datetime.datetime(y, 1, 1)#начало года
    ld=datetime.datetime(y+1, 1, 1)#конец года, не включая
    listOfDates=[date for date in np.arange(fd,ld,datetime.timedelta(days=1))]#лист с датами в этом году
    yd=fr.iloc[j+5:j+36,1:13]
    if y%4!=0:#не весокосный год, т.е. остаток от делления на 4 не равен 0:
        #ravel делает из 2d 1d строка за строкой, при этом где-то будут пустые ячейки т.к. не все месяцы по 31 день 
        d=np.delete(yd.values.transpose().ravel(),[59,60,61,123,185,278,340])
    else:
        d=np.delete(yd.values.transpose().ravel(),[60,61,123,185,278,340])
    r=pd.DataFrame(d, columns=nm, index=listOfDates)#делаем datarfame, который по сути series
    if fy==y:
        res[-1].append(r)
    #если пошел следующий год, лист с датафреймами из старого кода соеденяется, а новый датафрейм помещается в лист и 
    #становится основой для сбора расходов со следующего года
    else:
        res[-1]=pd.concat(res[-1], axis=1,sort=False)
        res.append([r])
        fy=y
#последний год объеденяется в единый фрейм уже вне тела цикла
res[-1]=pd.concat(res[-1], axis=1,sort=False)
#наконец все в месте соеденяется
res=pd.concat(res, axis=0,sort=False)

In [4]:
res=res.astype(str)#разные типы данных нам не к чему

In [5]:
#нб - не было в 0, а те ячейки где были дополнительные символы разделяем по пробелу и берем первую часть - число
def str_int(x):
    if x=="нб":
        return 0
    else:
        return float(x.replace(",",".").split(" ")[0])

In [6]:
res=res.applymap(str_int)# применяем функцию ко всем элементам фрейма

In [7]:
res

,75284
2008-01-01,26.7
2008-01-02,26.0
2008-01-03,26.0
2008-01-04,26.0
2008-01-05,25.6
...,...
2019-12-27,69.9
2019-12-28,75.2
2019-12-29,86.4
2019-12-30,87.2


In [8]:
df = res.copy()
df['date'] = df.index

In [9]:
df

,75284,date
2008-01-01,26.7,2008-01-01
2008-01-02,26.0,2008-01-02
2008-01-03,26.0,2008-01-03
2008-01-04,26.0,2008-01-04
2008-01-05,25.6,2008-01-05
...,...,...
2019-12-27,69.9,2019-12-27
2019-12-28,75.2,2019-12-28
2019-12-29,86.4,2019-12-29
2019-12-30,87.2,2019-12-30


In [10]:
df.reset_index(drop=True, inplace=True)
df['date'] = pd.to_datetime(df['date']).apply(lambda x: datetime.datetime.strftime(x, '%#d %#m %Y'))
# df.columns = ['Q', 'date']
df=df.astype(str)
df

,75284,date
0,26.7,1 1 2008
1,26.0,2 1 2008
2,26.0,3 1 2008
3,26.0,4 1 2008
4,25.6,5 1 2008
...,...,...
4378,69.9,27 12 2019
4379,75.2,28 12 2019
4380,86.4,29 12 2019
4381,87.2,30 12 2019


In [11]:
df2 = pd.DataFrame()
for i in df.columns:
    df2[i] = df["date"] + ' ' + df[i]
df2 = df2.drop(columns=['date'])

In [12]:
df2

,75284
0,1 1 2008 26.7
1,2 1 2008 26.0
2,3 1 2008 26.0
3,4 1 2008 26.0
4,5 1 2008 25.6
...,...
4378,27 12 2019 69.9
4379,28 12 2019 75.2
4380,29 12 2019 86.4
4381,30 12 2019 87.2


In [13]:
df2.to_excel('Унжа_для_продления.xlsx')

In [14]:
flat_gauge = [item for sublist in gauge_list for item in sublist]
flat_nm = [item for sublist in nm_list for item in sublist]


In [15]:
list(set(flat_gauge))

['Унжа - Р.УНЖА - Г.КОЛОГРИВ',
 'Унжа - р.Унжа - г.Кологрив',
 'Унжа - р. Унжа - г. Кологрив',
 'Унжа - р. Унжа - г.Кологрив']

In [16]:
list(set(flat_nm))

[75284]